In [414]:
import pandas as pd 
import numpy as np 
import torch 
import torch.nn as nn # torch.nn gets all the neural networks classes and functions
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset # Used to make PyTorch custom datasets
from sklearn.metrics import accuracy_score 

### Loading the dataset

In [415]:
df = pd.read_csv("./Datasets/diabetes.csv")
df.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


### Features & Labels Extraction 

In [416]:
# X : extract data from all rows and features column
# y : the label which classifies 
X = df.iloc[:,0:-1].values
y = np.array(df.iloc[:,-1])

In [417]:
for i in range(0,len(y)):
    if y[i] == 'positive':
        y[i] = 1
    if y[i] == 'negative' :
        y[i] = 0

In [418]:
y = np.array(y,dtype= 'float64')
type(X),type(y)

(numpy.ndarray, numpy.ndarray)

### Normalizing the features 

In [419]:
# Normalize the features in the range -1 to 1
sc = StandardScaler()
# x' = x - u / S.D (u = mean)
# Fit calculates the mean & S.D for the data and Transform applies that to the data

X = sc.fit_transform(X)

In [420]:
X = torch.tensor(X)
y = torch.tensor(y).unsqueeze(1)
X.type,y.type

(<function Tensor.type>, <function Tensor.type>)

### Building custom dataset

In [421]:
class Dataset(Dataset):

    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        return self.x[index],self.y[index]
    
    def __len__(self):
        return len(self.x)

In [422]:
dataset = Dataset(X,y)
len(dataset)

768

In [423]:
from torch.utils.data import DataLoader

In [424]:
# Loading the data to dataloader for batch processing and shuffling 
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                            batch_size =32,
                            shuffle = True)

In [425]:
print(f"there are {format(len(train_loader))} batches")

there are 24 batches


### Building the Neural Network

In [426]:
class Model(nn.Module):
    def __init__(self,input_features,output_features):
        super(Model,self).__init__()
        self.fc1 = nn.Linear(in_features=input_features,out_features=5)
        self.fc2 = nn.Linear(5,4)
        self.fc3 = nn.Linear(4,3)
        self.fc4 = nn.Linear(3,output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self,x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)

        return out
      

In [427]:
net = Model(7,1)

# Creating loss function
loss_fn = torch.nn.BCELoss(size_average=True,reduction='mean')

# setting up Optimizer
optimizer = torch.optim.SGD(net.parameters(),lr=0.1, momentum=0.9)

d:\Deep learning with Pytorch\_pytorch_\.venv\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


### Training the network

In [428]:
epochs = 250
for epoch in range(epochs):

    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()

        # Forwards propagation
        outputs = net(inputs)

        # calculate loss
        loss = loss_fn(outputs,labels)

        # Clear the gradient buffer 
        optimizer.zero_grad()

        # Backpropagation : calculating gradients 
        loss.backward()

        # Backpropagation : Updating weights
        optimizer.step()

    # Calculating accuracy 
    output = (outputs>0.5).float()
    acc = (output==labels).float().mean()

    print(f" {epoch+1}/{epochs} | Loss : {loss} | Accuracy : {acc}")



 1/250 | Loss : 0.5719203352928162 | Accuracy : 0.71875
 2/250 | Loss : 0.5396065711975098 | Accuracy : 0.71875
 3/250 | Loss : 0.4899923801422119 | Accuracy : 0.71875
 4/250 | Loss : 0.4203639030456543 | Accuracy : 0.8125
 5/250 | Loss : 0.38801565766334534 | Accuracy : 0.875
 6/250 | Loss : 0.39744076132774353 | Accuracy : 0.8125
 7/250 | Loss : 0.39927563071250916 | Accuracy : 0.8125
 8/250 | Loss : 0.45080679655075073 | Accuracy : 0.71875


 9/250 | Loss : 0.6585901379585266 | Accuracy : 0.625
 10/250 | Loss : 0.4342195987701416 | Accuracy : 0.8125
 11/250 | Loss : 0.4841436743736267 | Accuracy : 0.84375
 12/250 | Loss : 0.48975569009780884 | Accuracy : 0.75
 13/250 | Loss : 0.601681649684906 | Accuracy : 0.6875
 14/250 | Loss : 0.465107262134552 | Accuracy : 0.71875
 15/250 | Loss : 0.6128020882606506 | Accuracy : 0.6875
 16/250 | Loss : 0.3831137418746948 | Accuracy : 0.84375
 17/250 | Loss : 0.49536365270614624 | Accuracy : 0.6875
 18/250 | Loss : 0.42249807715415955 | Accuracy : 0.78125
 19/250 | Loss : 0.41516226530075073 | Accuracy : 0.78125
 20/250 | Loss : 0.5079939961433411 | Accuracy : 0.75
 21/250 | Loss : 0.5556071400642395 | Accuracy : 0.75
 22/250 | Loss : 0.4327622056007385 | Accuracy : 0.78125
 23/250 | Loss : 0.46872788667678833 | Accuracy : 0.65625
 24/250 | Loss : 0.4893897473812103 | Accuracy : 0.78125
 25/250 | Loss : 0.6479378342628479 | Accuracy : 0.625
 26/250 | Loss : 0.4416290521621704 | Accuracy

In [429]:
# Training accuracy 
with torch.inference_mode():
    preds = net(X.float())

In [430]:
preds = preds>0.5
preds == y.float()

tensor([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True

In [431]:
accuracy_final = accuracy_score(y.float(),preds)
accuracy_final

0.8346354166666666